please comments about making Seq2Seq models !

In [ ]:
#import
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
import datetime
from sklearn import preprocessing, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers import RepeatVector,TimeDistributed
from numpy import array
from keras.models import Sequential, load_model
import re
from tqdm import tqdm
import os
pd.set_option('max_columns', 50)

In [ ]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns
    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")
    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# データ
INPUT_DIR = '../input/m5-forecasting-accuracy'
train_sales = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv'))
calendar = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/calendar.csv'))
sellp = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/sell_prices.csv'))
sell_prices = reduce_mem_usage(sellp)

In [ ]:
time_series_columns = [f'd_{i}' for i in range(1, 1970)]
transfer_cal = pd.DataFrame(calendar[['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']].values.T, index=['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI'], columns= time_series_columns)
#欠損値を0で補完
transfer_cal = transfer_cal.fillna(0)
#なんか入ってたら10を入れる
event_name_1_se = transfer_cal.loc['event_name_1'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)
event_name_2_se = transfer_cal.loc['event_name_2'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)
#日にち変換
calendar['date'] = pd.to_datetime(calendar['date'])

#欠損値を補完
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    calendar[feature].fillna('unknown', inplace = True)
#エンコーディング
cat = ['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']
for feature in cat:
    encoder = preprocessing.LabelEncoder()
    calendar[feature] = encoder.fit_transform(calendar[feature])
    
transfer_cal = pd.DataFrame(calendar[['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']].values.T,index=['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI'])

#メインのデータとcalendarを結合した。
price_fea = calendar[['wm_yr_wk','date']].merge(sell_prices, on = ['wm_yr_wk'], how = 'left')
#各商品を一行にして提出ファイルと同じようにする
price_fea['id'] = price_fea['item_id']+'_'+price_fea['store_id']+'_validation'
#pivotでLSTMで使用できるように整える
df = price_fea.pivot(index='id',columns='date',values='sell_price')

#型合わせ
df = reduce_mem_usage(df)
price_fea = reduce_mem_usage(price_fea)

In [ ]:
#train_salesの商品数とdfを結合
#正規化も考慮
#後で正規化があってるか確認
price_df = train_sales.merge(df,on=['id'],how= 'left').iloc[:,1919:]
price_df.index = train_sales.id
price_df = price_df.fillna(0)
price_df["max"] = price_df.max(axis=1)
for com in [x for x in price_df.columns if (x!="max")]:
    price_df[com] = (price_df[com])/price_df["max"]
    
price_df = price_df.drop(["max"],axis=1)
price_df = reduce_mem_usage(price_df)

In [ ]:
#正規化も考慮
time_series_data = train_sales[[f'd_{i}' for i in range(1, 1913 + 1)]]
time_series_data["max"] = time_series_data.max(axis=1)
pur_max = time_series_data.max(axis=1)
for com in tqdm([x for x in time_series_data.columns if (x!="max")]):
    time_series_data[com] = (time_series_data[com])/time_series_data["max"]
    
time_series_data = time_series_data.drop(["max"],axis=1)
time_series_data = reduce_mem_usage(time_series_data)

In [ ]:
#正規化も考慮
transfer_cal["max"] = transfer_cal.max(axis=1)

for com in tqdm([x for x in transfer_cal.columns if (x!="max")]):
    transfer_cal[com] = (transfer_cal[com])/transfer_cal["max"]
    
transfer_cal = transfer_cal.drop(["max"],axis=1)
transfer_cal = reduce_mem_usage(transfer_cal)
transfer_cal.columns  = price_df.columns
transfer_cal.head()

In [ ]:
from datetime import timedelta
def daterange(_start, _end):
    for n in range((_end - _start).days):
        yield _start + timedelta(n)

#わかりにくいからcolumn名を日付に変える
start = pd.Timestamp(2011,1,29)
end   = pd.Timestamp(2016,4,25)

a = []
for i in daterange(start, end):
    a.append(i)

time_series_data.columns = a
time_series_data.head()
del a,start,end

In [ ]:
# d_1914~d_1941を予測するためにd_1913以降の最新のデータ100個を抜き出す
X_test_1 = []

TS_test_1 = time_series_data[time_series_data.columns[-100:]]
ran = [x for x in daterange(pd.Timestamp(2016,1,16),pd.Timestamp(2016,4,25))]
TF_test_1 = transfer_cal[ran]
PD_test_1 = price_df[ran]

for i in tqdm(range(time_series_data.shape[0])):
    X_test_1.append([list(t) for t in zip(TF_test_1.loc['event_name_1'],
                                   TF_test_1.loc['event_type_1'],
                                   TF_test_1.loc['event_name_2'],
                                   TF_test_1.loc['event_type_2'],
                                   TF_test_1.loc['snap_CA'],
                                   TF_test_1.loc['snap_TX'],
                                   TF_test_1.loc['snap_WI'],
                                   PD_test_1.iloc[i],
                                   TS_test_1.iloc[i][0:100],)])
                                         
X_test_1 = np.asarray(X_test_1, dtype=np.float32)

print(X_test_1.shape)

In [ ]:
# seq2seqの訓練データを作成する
# X_train_1 : 2015/12/19 ~ 2016/3/27 の100データ
# y_train_1 : 2016/3/28 ~ 28データ
import datetime

X_train_1 = []
y_train_1 = []

TS_1 = time_series_data[time_series_data.columns[-128:]]
ran = [x for x in daterange(pd.Timestamp(2015,12,19),pd.Timestamp(2016,3,28))]
TF_1 = transfer_cal[ran]
PD_1 = price_df[ran]


for i in tqdm(range(time_series_data.shape[0])):
    X_train_1.append([list(t) for t in zip(TF_1.loc['event_name_1'],
                                   TF_1.loc['event_type_1'],
                                   TF_1.loc['event_name_2'],
                                   TF_1.loc['event_type_2'],
                                   TF_1.loc['snap_CA'],
                                   TF_1.loc['snap_TX'],
                                   TF_1.loc['snap_WI'],
                                   PD_1.iloc[i],
                                   TS_1.iloc[i][0:100],)])

    y_train_1.append([list(t) for t in zip(TS_1.iloc[i][100:128])])
                                         
X_train_1 = np.asarray(X_train_1, dtype=np.float32)
y_train_1 = np.asarray(y_train_1, dtype=np.float32)

print(X_train_1.shape)
print(y_train_1.shape)

In [ ]:
# X_train_2 : 2015/11/22 ~ 2016/2/29 の100データ
# y_train_2 : 2016/3/28 ~ 28データ

X_train_2 = []
y_train_2 = []

ran = [x for x in daterange(pd.Timestamp(2015,11,22),pd.Timestamp(2016,3,1))]
TS_2 = time_series_data[ran]
TF_2 = transfer_cal[ran]
PD_2 = price_df[ran]
a = time_series_data[[x for x in daterange(pd.Timestamp(2016,3,1),pd.Timestamp(2016,3,29))]]


for i in tqdm(range(time_series_data.shape[0])):
    X_train_2.append([list(t) for t in zip(TF_2.loc['event_name_1'],
                                   TF_2.loc['event_type_1'],
                                   TF_2.loc['event_name_2'],
                                   TF_2.loc['event_type_2'],
                                   TF_2.loc['snap_CA'],
                                   TF_2.loc['snap_TX'],
                                   TF_2.loc['snap_WI'],
                                   PD_2.iloc[i],
                                   TS_2.iloc[i],)])

    y_train_2.append([list(t) for t in zip(a.iloc[i])])
                                         
X_train_2 = np.asarray(X_train_2, dtype=np.float32)
y_train_2 = np.asarray(y_train_2, dtype=np.float32)

print(X_train_2.shape)
print(y_train_2.shape)

In [ ]:
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
from keras.optimizers import *
import keras

def model(tr,te,va_tr):
    num_encoder_tokens = tr.shape[1]
    num_decoder_tokens = 28
    #お試しで中間層は28
    latent_dim = 32

    # <encoder>
    encoder_inputs = Input(shape=(None,8),name='encoder_input_1')
    e_i =BatchNormalization(axis=-1)(encoder_inputs)
    #RNNレイヤーを複数積み重ねたい時は、各時刻で層間のデータのやり取りがあるので、(少なくとも最後の層以外は)必ずTrueにしなければならないようです。
    encoder_out_1, state_h_1, state_c_1 = LSTM(latent_dim, return_state=True,return_sequences=True,name='encoder_LSTM_1')(e_i)
    encoder_out_2, state_h_2, state_c_2 = LSTM(latent_dim, return_state=True,return_sequences=True,name='encoder_LSTM_2')(encoder_out_1)
    encoder_states = [state_h_2, state_c_2]
    # のちに使うのでencode_modelを作成しておく
    encoder_model = Model(encoder_inputs,encoder_states)
    #encoder_model.summary()

    # <decoder>
    decoder_inputs = Input(shape=(None,1),name='decoder_input_1')  
    d_i=BatchNormalization(axis=-1)(decoder_inputs)
    decoder_out_1, de_state_h_1, de_state_c_1 = LSTM(latent_dim, return_state=True,return_sequences=True,name='decoder_LSTM_1')(d_i,initial_state=encoder_states)
    decoder_out_2, de_state_h_2, de_state_c_2 = LSTM(latent_dim, return_state=True,return_sequences=True,name='decoder_LSTM_2')(decoder_out_1)
    decoder_outputs = Dense(1,activation='sigmoid',name='output')(decoder_out_2)
    
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
    #model.summary()
    model.compile(optimizer=Adam(lr=0.0001,beta_1=0.9), loss='mse')
    
    # model訓練
    # seq2seqでは、decoder訓練に「一つ前の時刻データ」を入力し、「現時刻のデータ」出力するように学習する
    model.fit([tr[:,:,:8],np.pad(te[:,1:,:],[(0,0),(1,0),(0,0)])],te,epochs=2, batch_size=5000,verbose=1)
    
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True,return_sequences=True,name='decoder_LSTM_2')(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = Dense(1,activation='sigmoid',name='output')(decoder_outputs)

    decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    # <予測>
    states_value = encoder_model.predict(va_tr[:,:,:8])
    target_seq = np.zeros((30490,1,1))
    target_seq[[0, 0, 1]]= 1.

    for i in range(0,28):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Update states
        states_value = [h, c]
        if i==0:
            a = output_tokens
        else:
            a = np.concatenate([a, output_tokens], 1)

        target_seq = output_tokens
    
    
    return a

In [ ]:
# まず第一フェーズの予測をする
pred = model(X_train_2,y_train_2,X_train_1)
from sklearn.metrics import mean_absolute_error
print("mas : {}".format(mean_absolute_error(pred.reshape([30490*28,1]),y_train_1.reshape([30490*28,1]))))

validation_prediction = model(X_train_1,y_train_1,X_test_1)

In [ ]:
# 第一フェーズの予測を組み合わせて、第二フェーズで使用すべきデータを準備する

# eva_test : ~ d_1913　の最新データ100個

X_test_2 = []
y_test_2 = []
eva_test = []

for i in tqdm(range(time_series_data.shape[0])):
    X_test_2.append([list(t) for t in zip(transfer_cal.loc['event_name_1'],
                                   transfer_cal.loc['event_type_1'][-128:-28],
                                   transfer_cal.loc['event_name_2'][-128:-28],
                                   transfer_cal.loc['event_type_2'][-128:-28],
                                   transfer_cal.loc['snap_CA'][-128:-28],
                                   transfer_cal.loc['snap_TX'][-128:-28],
                                   transfer_cal.loc['snap_WI'][-128:-28],
                                   price_df.iloc[i][-128:-28],)])
    
    y_test_2.append([list(t) for t in zip(time_series_data.iloc[i][-72:])])
    
    eva_test.append([list(t) for t in zip(transfer_cal.loc['event_name_1'],
                                   transfer_cal.loc['event_type_1'][-28:],
                                   transfer_cal.loc['event_name_2'][-28:],
                                   transfer_cal.loc['event_type_2'][-28:],
                                   transfer_cal.loc['snap_CA'][-28:],
                                   transfer_cal.loc['snap_TX'][-28:],
                                   transfer_cal.loc['snap_WI'][-28:],
                                   price_df.iloc[i][-28:],)])
                                         
X_test_2 = np.asarray(X_test_2, dtype=np.float32)               
y_test_2 = np.asarray(y_test_2, dtype=np.float32)
eva_test = np.asarray(eva_test, dtype=np.float32) 

y_test_2 = np.concatenate([y_test_2,validation_prediction],1)

print(X_test_2.shape)
print(y_test_2.shape)
print(eva_test.shape)

In [ ]:
#第2フェーズの予測をする
evaluation_prediction = model(X_test_2,y_test_2,eva_test)

In [ ]:
# 第一フェースのデータをまとめる
validation = pd.DataFrame(validation_prediction.astype(np.float32).reshape(X_train_1.shape[0],28))
validation["max"] = pur_max

for com in [x for x in validation.columns if x!="max"]:
    validation[com] = validation[com]*validation["max"]
    validation[com] = validation[com].apply(lambda x:np.rint(x))

validation = validation.drop("max",axis=1) 
validation.columns = ["F{}".format(x) for x in range(1,29)]
validation.insert(0,"id",price_df.index)
validation.head()

In [ ]:
# 第二フェースのデータをまとめる
evaluation = pd.DataFrame(evaluation_prediction.astype(np.float32).reshape(X_train_1.shape[0],28))
evaluation["max"] = pur_max

for com in [x for x in evaluation.columns if x!="max"]:
    evaluation[com] = evaluation[com]*evaluation["max"]
    evaluation[com] = evaluation[com].apply(lambda x:np.rint(x))

evaluation = evaluation.drop("max",axis=1) 
evaluation.columns = ["F{}".format(x) for x in range(1,29)]
evaluation.insert(0,"id",[x.replace("validation",'evaluation') for x in price_df.index])
evaluation.head()

In [ ]:
#提出
submission = validation.append(evaluation)
submission.to_csv('submission.csv', index=False)
submission.head()